## noisy한 데이터를 ascii 코드 기준으로 텍스트의 0.x 이상 

In [2]:
import pandas as pd
import os
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [3]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, 'outputs')

# labeled_data_path = os.path.join(DATA_DIR, "noised_label")

In [25]:
df_list = []
threshold = 0.2
df = pd.read_csv(DATA_DIR+f"/train_modified_t{threshold}_ver2.csv")
df_list.append(df)
print(df.head())

                    ID                              text  target  ascii_ratio  \
0  ynat-v1_train_00000  정^ ^파^ 미사^ ^^^ 이용기간 ^^ 단^ ^분종^^보       4     0.406250   
1  ynat-v1_train_00001       ^찰^국^^^^ 로^^한^^ 회장 ^ ^^^^송^       3     0.592593   
2  ynat-v1_train_00002            ^ 김정^ 자주통일 새^^^열^나가야^보       2     0.318182   
3  ynat-v1_train_00003     갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩       5     0.103448   
4  ynat-v1_train_00004      ^^美대선^앞두고 ^^^^단 발^ ^비해 감시 강화       6     0.321429   

   is_noisy  
0         1  
1         1  
2         1  
3         0  
4         1  


In [6]:
test_text = df_list[0]['text'][3]
test_text

'갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩'

In [7]:
# 모델과 토크나이저 로드
model_name = 'Saxo/Linkbricks-Horizon-AI-Korean-Pro-8B'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto")

# 노이즈가 있는 뉴스 기사 제목
noisy_title = "세계K인무역협회 올S2 무역인 1천b55w 배출한"


tokenizer_config.json:   0%|          | 0.00/57.9k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/462 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/913 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/22.8k [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.19G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


In [9]:
# 프롬프트 작성
prompt1 = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

노이즈가 포함된 제목: '{noisy_title}'

복원 시 다음 지침을 따라주세요:
1. 무작위로 삽입된 영문자와 숫자를 제거하세요.
2. 특수문자를 적절히 처리하세요.
3. 줄임말이나 약어는 가능한 원래 형태로 복원하세요.
4. 문맥을 고려하여 누락된 단어나 조사를 추가하세요.
5. 제목의 전체적인 의미를 유지하면서 자연스러운 한국어 문장으로 만드세요.

복원된 제목:"""

prompt2 = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

다음은 노이즈가 제거된 예시입니다:

1. 노이즈가 포함된 제목: '게시판 KISA 박민정 책임연구원 APTLD 이사 선출'
   복원된 제목: 'KISA 박민정 책임연구원, APTLD 이사 선출'

2. 노이즈가 포함된 제목: 'UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아'
   복원된 제목: 'UEFA 챔스리그 4강, 마드리드 더비 성사... 레알 vs 아틀레티코'

3. 노이즈가 포함된 제목: '본t카 애스턴마틴 Sp라 국내 4P500대g 생산'
   복원된 제목: '본드카 애스턴마틴 스포츠카, 국내 450대 생산'

이제 다음 제목의 노이즈를 제거해주세요:

노이즈가 포함된 제목: '{noisy_title}'

복원된 제목:"""

prompt3 = f"""당신은 다국어 능력을 갖춘 전문 번역 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목을 정제하기 위해 다음 단계를 수행해주세요:

1. 노이즈가 포함된 한국어 제목을 영어로 번역하세요. 이 때, 노이즈로 보이는 부분은 최대한 문맥을 고려하여 자연스럽게 해석하세요.
2. 번역된 영어 제목을 다시 한국어로 번역하세요. 이 과정에서 원래 제목의 의미를 최대한 보존하면서 자연스러운 한국어 뉴스 제목을 만들어주세요.

참고: 다음은 이 과정의 예시입니다.

예시 1:
노이즈가 포함된 한국어 제목: '게시판 KISA 박민정 책임연구원 APTLD 이사 선출'
1단계 - 영어 번역: 'KISA Senior Researcher Park Min-jung Elected as APTLD Director'
2단계 - 한국어 재번역: 'KISA 박민정 책임연구원, APTLD 이사로 선출'

예시 2:
노이즈가 포함된 한국어 제목: 'UEFA 챔스리그 4강 마드리드 더비 성사레알 vs 아'
1단계 - 영어 번역: 'UEFA Champions League Semifinals: Madrid Derby Confirmed, Real vs Atletico'
2단계 - 한국어 재번역: 'UEFA 챔피언스리그 4강: 마드리드 더비 성사, 레알 vs 아틀레티코'

이제 주어진 제목에 대해 위 과정을 수행해주세요.
노이즈가 포함된 제목: '{noisy_title}'

복원된 제목:"""

# 토크나이즈 및 모델 입력 생성
input1 = tokenizer(prompt1, return_tensors="pt").to(model.device)
input2 = tokenizer(prompt2, return_tensors="pt").to(model.device)
input3 = tokenizer(prompt3, return_tensors="pt").to(model.device)
# 모델을 사용하여 텍스트 생성
with torch.no_grad():
    outputs1 = model.generate(
        **input1,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

    outputs2 = model.generate(
        **input2,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

    outputs3 = model.generate(
        **input3,
        max_new_tokens=50,
        num_return_sequences=1,
        temperature=0.7,
        top_p=0.95,
        do_sample=True
    )

# 생성된 텍스트 디코딩
generated_text1 = tokenizer.decode(outputs1[0], skip_special_tokens=True)
generated_text2 = tokenizer.decode(outputs2[0], skip_special_tokens=True)
generated_text3 = tokenizer.decode(outputs3[0], skip_special_tokens=True)

# 결과 출력
print("model_name : ", model_name)
print("original text : ", noisy_title)
print("(step_prompt_result) : ", generated_text1.split("복원된 제목:")[-1].strip())
print("------------")
print("(few_shot_result) : ",generated_text2.split("복원된 제목:")[-1].strip())
print("------------")
print("(back_translation) : ",generated_text3.split("복원된 제목:")[-1].strip())

Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


model_name :  Saxo/Linkbricks-Horizon-AI-Korean-Pro-8B
original text :  세계K인무역협회 올S2 무역인 1천b55w 배출한
(step_prompt_result) :  '세계K인무역협회 올해 무역인 1천명 배출한'

설명:
1. 'S2'와 '1천b55w'는 무작위로 삽입된 영문
------------
(few_shot_result) :  '세계K인무역협회 올해 S2 무
------------
(back_translation) :  '세계K인무역협회 올S2 무역인 1천명 배출한' (노이즈 제거 및 문맥 유지)

번역 과정:
1단계 - 영어 번역: 'World K Trade


In [10]:
df_list[0]

,ID,text,target,ascii_ratio,is_noisy
0,ynat-v1_train_00000,정 파 미사 이용기간 단 분종 보,4,0.625000,1
1,ynat-v1_train_00001,찰 국 로 한 회장 송,3,0.740741,1
2,ynat-v1_train_00002,김정 자주통일 새 열 나가야 보,2,0.454545,1
3,ynat-v1_train_00003,갤노트8 주말 27만대 개통…시장은 불법 보조금 얼룩,5,0.310345,0
4,ynat-v1_train_00004,美대선 앞두고 단 발 비해 감시 강화,6,0.500000,1
...,...,...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨…10점만점에 12점도,6,0.272727,0
2796,ynat-v1_train_02796,삼성 갤럭시S9 정식 출시 첫 주말 이통시장 잠잠,2,0.333333,0
2797,ynat-v1_train_02797,텔레그램 한 등 亞서 시간 다운… 버 정 종 보,4,0.468750,1
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해…승부는 냉정,1,0.225806,0


In [20]:
"""
선정된 모델을 이용해 기존의 noise dataset의 텍스트 중 노이즈로 판별된 데이터만 복구하는 코드 (step-prompting)
"""
import re 

def extract_quoted_string(text):
    pattern = r"'([^']*)'"  # 작은따옴표 사이의 문자열을 찾는 패턴
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]

for idx, df in enumerate(df_list) : 
    temp_df = df.copy()
    for tidx, (noisy_text, is_noisy) in tqdm(enumerate(zip(df['text'], df['is_noisy']))) :
        if tidx ==2 : break
        if is_noisy == 0 :
          continue
        prompt = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 공백으로 치환된 한국어 뉴스 기사의 제목입니다. 이 제목에서 원래의 자연스러운 뉴스 제목으로 복원해주세요.

        노이즈가 포함된 제목: '{noisy_text}'

        복원 시 다음 지침을 따라주세요:
        1. 이 문장에서 의미를 자연스럽게 만들기 위해 공백이 들어가야 할 위치를 분석하세요.
        2. 이 문장의 공백을자연스럽게 채울 수 있는 한국어 단어를 3~5개 정도 제안해 주세요.
        3. 이 문장의 공백에 [단어]를 넣어 자연스러운 문장으로 완성해 주세요.
        4. 완성된 문장이 자연스러운지 검토하고, 더 적절한 단어로 수정할 부분이 있다면 수정해 주세요.

        복원된 제목:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = extract_quoted_string(generated_text.split("복원된 제목:")[-1].strip())
        temp_df.loc[tidx, 'text'] = cleaned_text
        print("origin : ", noisy_text)
        print("cleaned_text : ", cleaned_text)

    temp_df.to_csv(f'../data/recovery_data/c1_label_{idx}.csv', index=False)

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
1it [00:18, 18.13s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  정   파  미사      이용기간    단   분종  보
cleaned_text :  정파 미사용 기간 단분 종보


2it [00:34, 17.50s/it]

origin :   찰 국     로  한   회장       송 
cleaned_text :  찰스 국로 한 회장 송


In [51]:
"""
선정된 모델을 이용해 기존의 noise dataset의 텍스트 중 노이즈로 판별된 데이터만 복구하는 코드 (noise는 ^로 변환환, few-shot)
"""
import re 

def extract_quoted_string(text):
    pattern = r"'([^']*)'"  # 작은따옴표 사이의 문자열을 찾는 패턴
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]

for idx, df in enumerate(df_list) : 
    temp_df = df.copy()
    for tidx, (noisy_text, is_noisy) in tqdm(enumerate(zip(df['text'], df['is_noisy']))) :
        if tidx == 20 : break
        if is_noisy == 0 :
          continue
        prompt = f"""당신은 전문적인 한국어 텍스트 복원 AI입니다. 다음은 노이즈가 "^"로 치환된 한국어 뉴스 기사의 제목입니다. 이 제목에서 원래의 자연스러운 뉴스 제목으로 복원해주세요.


        다음은 노이즈를 원래의 문자로 복구하여 자연스러운 문장을 생성한 예시입니다:

        1. 노이즈가 포함된 제목: '^^^레콤 ^분기 영^익^^천^^^^…^^^％ 증가'
        복원된 제목: 'SK텔레콤 4분기 영업익 오천오백육억...200% 증가'

        2. 노이즈가 포함된 제목: '감사원 수^과학^구소^연구보다^워크^·강연'
        복원된 제목: '감사원 수리과학연구소 연구보다 워크샵·강연'

        3. 노이즈가 포함된 제목: '^^∼^^대는 유튜브 세대…하루 ^^^회 ^^분 본다'
        복원된 제목: '10∼20대는 유튜브 세대…하루 100회 90분 본다'

        4. 노이즈가 포함된 제목: '^ 김정^ 자주통일 새^^^열^나가야^보'
        복원된 제목: '北 김정은 자주통일 새 시대 열어 나가야 1보 '
        
        위의 예시와 같이 복원할 때에는 다음과 같은 규칙이 존재합니다.
        1. ^ 문자가 한 번만 등장하면 그 자리에 적합한 한 글자를 넣어 자연스럽게 복원합니다.
        2. ^ 문자가 여러 번 연속으로 등장하면, ^의 개수를 유지하면서 그 자리에 필요한 단어를 넣습니다.
        
        이제 다음 제목의 노이즈를 문장에 어울리는 단어로 치환하여 자연스러운 문장으로 만들어주세요:

        노이즈가 포함된 제목: '{noisy_text}'

        복원된 제목:"""
        ##         1. "^"와 공백이 아닌 기존 한글 문자는 그대로 둔 채 최대한 나머지 문자를 채워주세요.
        ##         2. "^"는 무조건 "^"와 공백이 아닌 한 글자로 치환해주세요
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = extract_quoted_string(generated_text.split("복원된 제목:")[-1].strip())
        temp_df.loc[tidx, 'text'] = cleaned_text
        # print("generated_text : ", generated_text)
        print("origin : ", noisy_text)
        print("cleaned_text : ", cleaned_text)

    temp_df.to_csv(f'../data/recovery_data/converted_train_ver2_fewshot.csv', index=False)

0it [00:00, ?it/s]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
1it [00:17, 17.72s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  정^ ^파^ 미사^ ^^^ 이용기간 ^^ 단^ ^분종^^보
cleaned_text :  정파 미사일 ^^^ 이용기간 ^ 단 ^ 분종 ^보


2it [00:35, 17.69s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  ^찰^국^^^^ 로^^한^^ 회장 ^ ^^^^송^
cleaned_text :  찰스 국 로한 회장 승송


3it [00:52, 17.61s/it]Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  ^ 김정^ 자주통일 새^^^열^나가야^보
cleaned_text :  김정은 자주통일 새 시대 열어 나가야 1보 


In [ ]:
"""
선정된 모델을 이용해 기존의 noise dataset의 텍스트 중 노이즈로 판별된 데이터만 복구하는 코드 (noise는 ^로 변환환)
"""
import re 

def extract_quoted_string(text):
    pattern = r"'([^']*)'"  # 작은따옴표 사이의 문자열을 찾는 패턴
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]

for idx, df in enumerate(df_list) : 
    temp_df = df.copy()
    for tidx, (noisy_text, is_noisy) in tqdm(enumerate(zip(df['text'], df['is_noisy']))) :
        if tidx ==2 : break
        if is_noisy == 0 :
          continue
        prompt = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 공백으로 치환된 한국어 뉴스 기사의 제목입니다. 이 제목에서 원래의 자연스러운 뉴스 제목으로 복원해주세요.

        노이즈가 포함된 제목: '{noisy_text}'

        복원 시 다음 지침을 따라주세요:
        1. 이 문장에서 의미를 자연스럽게 만들기 위해 공백이 들어가야 할 위치를 분석하세요.
        2. 이 문장의 공백을자연스럽게 채울 수 있는 한국어 단어를 3~5개 정도 제안해 주세요.
        3. 이 문장의 공백에 [단어]를 넣어 자연스러운 문장으로 완성해 주세요.
        4. 완성된 문장이 자연스러운지 검토하고, 더 적절한 단어로 수정할 부분이 있다면 수정해 주세요.

        복원된 제목:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = extract_quoted_string(generated_text.split("복원된 제목:")[-1].strip())
        temp_df.loc[tidx, 'text'] = cleaned_text
        print("origin : ", noisy_text)
        print("cleaned_text : ", cleaned_text)

    temp_df.to_csv(f'../data/recovery_data/c1_label_{idx}.csv', index=False)

In [ ]:
"""
선정된 모델을 이용해 기존의 noise dataset을 처리하는 코드 2 
"""
import re

def extract_quoted_string(text):
    pattern = r"'([^']*)'"  # 작은따옴표 사이의 문자열을 찾는 패턴
    matches = re.findall(pattern, text)
    if len(matches) == 0 :
        return "제목없음"
    else :
        return matches[0]


for idx, df in enumerate(df_list) : 
    temp_df = df.copy()
    for tidx, noisy_text in tqdm(enumerate(df['text'])) :
        prompt = f"""당신은 전문적인 한국어 텍스트 정제 AI입니다. 다음은 노이즈가 포함된 한국어 뉴스 기사의 제목입니다. 이 제목에서 노이즈를 제거하고 원래의 자연스러운 뉴스 제목으로 복원해주세요.

        노이즈가 포함된 제목: '{noisy_text}'

        복원 시 다음 지침을 따라주세요:
        1. 무작위로 삽입된 영문자와 숫자를 제거하세요.
        2. 특수문자를 적절히 처리하세요.
        3. 줄임말이나 약어는 가능한 원래 형태로 복원하세요.
        4. 문맥을 고려하여 누락된 단어나 조사를 추가하세요.
        5. 제목의 전체적인 의미를 유지하면서 자연스러운 한국어 문장으로 만드세요.

        복원된 제목:"""

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=50,
                num_return_sequences=1,
                temperature=0.7,
                top_p=0.95,
                do_sample=True
            )

        generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
        cleaned_text = extract_quoted_string(generated_text.split("복원된 제목:")[-1].strip())
        temp_df.loc[tidx, 'text'] = cleaned_text
        print("origin : ", noisy_text)
        print("cleaned : ", cleaned_text)

    temp_df.to_csv(f'./data/cleaning_step1/c1_label_{idx}.csv', index=False)